# 邮件发送

In [ ]:
import os
import smtplib
import time
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.header import Header
from logger import Logging

logname = os.path.splitext(os.path.basename(__file__))[0]
logger = Logging(logname)
log = logger.logObject


def send_mail(sender, passwd, title, text, receiver, cc_receiver, filename=None):
    # 创建一个带附件的实例
    message = MIMEMultipart()
    message['From'] = Header(sender)
    message['To'] = Header(', '.join(receiver), 'utf-8')
    message['Cc'] = Header(', '.join(cc_receiver), 'utf-8')
    message['Subject'] = Header(title, 'utf-8')
    message.attach(MIMEText(text, 'html', 'utf-8'))
    if filename is not None:
        # 构造附件1，传送当前目录下的 test.txt 文件
        att1 = MIMEText(open(filename, 'rb').read(), 'base64', 'utf-8')
        att1["Content-Type"] = 'application/octet-stream'
        # 这里的filename可以任意写，写什么名字，邮件中显示什么名字
        att1["Content-Disposition"] = 'attachment; filename="{}"'.format(filename)
        message.attach(att1)

    for i in range(5):
        try:
            smtpObj = smtplib.SMTP()
            smtpObj.connect('mail.163.com', 587)
            smtpObj.starttls()
            smtpObj.login(sender, passwd)
            smtpObj.sendmail(sender, receiver + cc_receiver,
                             message.as_string())
            log.info("邮件发送成功")
            break
        except smtplib.SMTPException as e:
            log.error(e)
            log.error("Error: 无法发送邮件,程序正在重试，请勿关闭！")
            time.sleep(5 * i)
